In [15]:
from pseudo_povm_circuits import *
import cmath
import math
import random

In [2]:
def generate_random_pseudo_povm():
  '''Generates random pseudo POVM

  Returns instance of PseudoPovm class
  '''
  
  povm_ten = tf.random.normal([4, 3], dtype = tf.float64)
  return PseudoPovm(povm_ten)

In [3]:
def total_negativity_optimization(circuit):
  '''Finds pseudo POVM where total negativity of given circuit is minimal

  Arguments
  ---------
  circuit : QubitCircuit 

  Returns instance of class PseudoPovm
  '''


  #=========================#
  lr = 0.02 # learning rate
  #=========================#
  opt = tf.keras.optimizers.Adam(lr)

  def neg(var):
    povm = PseudoPovm(var)
    return circuit.total_negativity(povm) 

  povm = generate_random_pseudo_povm()


  var = povm.lam
  var = tf.Variable(var)


  #print(neg(var))

  for i in range(1000):
    with tf.GradientTape() as tape:

      loss = neg(var)
    grad = tape.gradient(loss, var) 
    opt.apply_gradients(zip([grad], [var]))  # minimization step
  
  lam = tf.complex(var,tf.zeros([4, 3], dtype = tf.float64))
  return PseudoPovm(lam)

In [4]:
# Initialize some single qubit gates
X = sigma_x
Y = sigma_y
Z = sigma_z
T = tf.constant([[1 ,0],[0, cmath.exp(1j * math.pi / 4)]], dtype=tf.complex128)
H = tf.constant([[1,1],[1,-1]], dtype=tf.complex128) / math.sqrt(2)
S = tf.constant([[1,0],[0,1j]],dtype=tf.complex128)

In [14]:
#In this example we consider only gates from clifford group
clifford_group = [T,S,H]

In [75]:
# Create random sequence of gates from cliffor group of length gates_num
gates_num = 10
gates = []
for i in range(gates_num):
  gates.append(random.choice(clifford_group))

In [76]:
# Create pseudo POVM
povm = generate_random_pseudo_povm()

In [77]:
# Create quantum circuit
circuit = QubitCircuit(gates)

In [78]:
# Run circuit 
circuit.run_circuit(povm)

<tf.Tensor: shape=(2,), dtype=float64, numpy=array([0.5, 0.5])>

In [80]:
circuit.run_approx_circuit(povm)

<tf.Tensor: shape=(2,), dtype=float64, numpy=array([0., 1.])>

In [81]:
# Check total negativity before optimization
circuit.total_negativity(povm)

<tf.Tensor: shape=(), dtype=float64, numpy=58.61914554325796>

In [82]:
# Find pseudo POVM where negativity is minimal
optimal_basis = total_negativity_optimization(circuit)

In [83]:
# Let's check total negativity in nev pseudo POVM
circuit.total_negativity(optimal_basis)

<tf.Tensor: shape=(), dtype=float64, numpy=3.3239140665998623>

In [84]:
circuit.run_circuit(optimal_basis)

<tf.Tensor: shape=(2,), dtype=float64, numpy=array([0.5, 0.5])>

In [85]:
circuit.run_approx_circuit(optimal_basis)

<tf.Tensor: shape=(2,), dtype=float64, numpy=array([0.53645158, 0.46354842])>